# VacationPy
### Moriah Taylor
#### April 24, 2021

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import API key
from config import api_key

### Store Part I results into DataFrame

In [2]:
#specify path
path = "../WeatherPy/output_data/cities.csv"
#import csv file from part 1
city_df = pd.read_csv(path)

### Humidity Heatmap

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=api_key)

In [4]:
#store lat and lng into locations
locations = city_df[['Lat','Lng']].astype(float)
#convert humidity to float
humidity = city_df['Humidity'].astype(float)

#heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
#drop null values (shouldn't be any)
reduced_df = city_df.dropna(axis=0, how='all')
#subset to weather conditions
##max temp lower than 80
##max temp greater than 70
##wind speed less than 10
##zero cloudiness
reduced_df = reduced_df[(reduced_df['Max Temp']<80) & (reduced_df['Max Temp']>70) & (reduced_df['Wind Speed']<10) & (reduced_df['Cloudiness']==0)]    
reduced_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
454,santa cruz cabralia,-16.2781,-39.0247,78.8,69,0,6.91,BR,1619221082
459,haimen,23.1935,116.6142,73.4,73,0,4.47,CN,1619221085
472,najran,17.4924,44.1277,71.6,24,0,2.68,SA,1619221088
496,sakakah,29.9697,40.2064,77.0,13,0,9.22,SA,1619221094


### Hotel Map

In [9]:
#create hotel df
hotel_df = reduced_df[['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ""

In [15]:
# find the closest hotel within 5000 meters of city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create for loop to go through each row
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "key": api_key
    }
    
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    print(f"Results for index {i}: {city}.")
    
    try:
        print(places_data["results"][0]['name'])
        hotel_df.loc[i, "Hotel Name"] = places_data["results"][0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[i, "Hotel Name"] = "NA"
        print("Results not found.")
        
    print("----------")
print("-------Done--------")

Results for index 454: santa cruz cabralia.
Campo Bahia
----------
Results for index 459: haimen.
Results not found.
----------
Results for index 472: najran.
Results not found.
----------
Results for index 496: sakakah.
Raoum Inn Hotel
----------
-------Done--------


In [22]:
#capitalize city names to make better-looking markers
hotel_df['City'] = ['Santa Cruz Cabralia', 'Haimen', 'Najran', 'Sakakah']  #easier/faster than doing str.split() and str.capitalize()
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
454,Santa Cruz Cabralia,BR,-16.2781,-39.0247,Campo Bahia
459,Haimen,CN,23.1935,116.6142,NaN
472,Najran,SA,17.4924,44.1277,NaN
496,Sakakah,SA,29.9697,40.2064,Raoum Inn Hotel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))